In [22]:
import math
import random
import folium
import google_apis
import numpy as np
import pandas as pd

In [23]:
def calc_filter_window(route_points, search_distance):
    # search_distance in kms
    lats = [point[0] for point in route_points]
    lons = [point[1] for point in route_points]
    rt_min_lat = min(lats)
    rt_max_lat = max(lats)
    rt_min_lon = min(lons)
    rt_max_lon = max(lons)

    lon_offset= (search_distance * 360) / (2 * math.pi * 6371.0 * math.cos(math.radians(rt_max_lat)))
    lat_offset = search_distance/111.0

    lat_window = rt_min_lat - lat_offset, rt_max_lat + lat_offset
    lon_window = rt_min_lon - lon_offset, rt_max_lon + lon_offset

    ne_point = rt_max_lat + lat_offset, rt_max_lon + lon_offset
    sw_point = rt_min_lat - lat_offset, rt_min_lon - lon_offset

    return sw_point, ne_point

def calculate_zoom_level(lat1, lon1, lat2, lon2, width_pixels):
    R = 6371  # Earth's radius in kilometers
    B = width_pixels  # Width of the map window in pixels

    delta_lon = abs(lon1 - lon2)
    lat_rad = math.radians(lat1)
    zoom = math.log2(360 * ((2 * R * math.cos(lat_rad) * math.pi) / (256 * B * delta_lon)))

    return int(zoom)

def haversine_distance(pt1, pt2):
    r = 6371.0  # Earth's radius in kilometers

    lat1, lon1 = pt1
    lat2, lon2 = pt2

    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)

    delta_lat = lat2_rad - lat1_rad
    delta_lon = lon2_rad - lon1_rad

    a = math.sin(delta_lat / 2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(delta_lon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    distance = r * c  # Distance in kilometers
    return distance

def range_calc(ddd_row, rdm_rt_pt):
    ddd_lat = ddd_row['latitude']
    ddd_lon = ddd_row['longitude']
    return haversine_distance(rdm_rt_pt, (ddd_lat, ddd_lon))

def package_route_points(route_points):
    route_points_df = pd.DataFrame(route_points, columns=['latitude', 'longitude'])
    route_df = route_points_df[['longitude', 'latitude']].apply(list, axis=1).to_frame()
    route_df['num'] = 0
    route_df = route_df.groupby('num').agg(**{
        'path': pd.NamedAgg(column=0, aggfunc=list)
    })
    return route_df

In [24]:
all_ddd_locations = pd.read_csv('ddd_locations.csv', na_values=['ERROR', 'ERROR, ERROR, ERROR ERROR']).dropna()
ddds_in_range = all_ddd_locations
route_points = []

start_address = 'nyc'
end_address = 'la'
search_distance = 10

# if start_address and end_address:
route_points = google_apis.get_route_points(start_address, end_address)
    # if len(route_points) > 0:
filter_window = calc_filter_window(route_points, search_distance)

In [6]:
print(route_points[0:5])
print(filter_window)
ddds_in_range.sample(7)

[(40.71268, -74.00658), (40.76357, -74.02479), (40.81084, -74.0855), (40.8853, -74.21434), (40.86328, -74.33531)]
((33.964819909909906, -118.36320375893047), (41.84570009009009, -73.88602624106953))


,loc_name,address,city,state,zip,url,full_address,latitude,longitude
921,LAKE EFFECT DINER,3165 Main St,Buffalo,NY,14214,https://www.dinersdriveinsdiveslocations.com/l...,"3165 Main St, Buffalo, NY 14214",42.950291,-78.826540
486,BIG WAVE SHRIMP TRUCK,66-521 Kamehameha Hwy.,Haleiwa,HI,96712,https://www.dinersdriveinsdiveslocations.com/b...,"66-521 Kamehameha Hwy., Haleiwa, HI 96712",21.579487,-158.105483
1133,FUEL CANTINA,211 Rutledge Ave.,Charleston,SC,29403,https://www.dinersdriveinsdiveslocations.com/f...,"211 Rutledge Ave., Charleston, SC 29403",32.789307,-79.947226
1033,EISCHEN'S BAR & GRILL,108 2nd South St.,Okarche,OK,73762,https://www.dinersdriveinsdiveslocations.com/e...,"108 2nd South St., Okarche, OK 73762",35.725103,-97.975093
551,DELL RHEA'S CHICKEN BASKET,645 Joliet Road,Willowbrook,IL,60527,https://www.dinersdriveinsdiveslocations.com/d...,"645 Joliet Road, Willowbrook, IL 60527",41.745537,-87.939263
874,JERSEY GIRL CAFE,608 Marketplace Blvd,Hamilton,NJ,08691,https://www.dinersdriveinsdiveslocations.com/j...,"608 Marketplace Blvd, Hamilton, NJ 08691",40.195268,-74.636618
390,GARBO'S GRILL FOOD CART,603 Greene St,Key West,FL,33040,https://www.dinersdriveinsdiveslocations.com/g...,"603 Greene St, Key West, FL 33040",24.559902,-81.803972


In [ ]:
##############################################################################################

In [19]:
# find_ddds_along_route(ddds, route_points, filter_window, search_distance)

ddds = all_ddd_locations#.drop(columns=['address', 'city', 'state', 'zip', 'url']).sample(20)################################
sw_point, ne_point = filter_window
coord_filter = (ddds['latitude'] > sw_point[0]) & (ddds['latitude'] < ne_point[0]) &\
               (ddds['longitude'] > sw_point[1]) & (ddds['longitude'] < ne_point[1])

ddds['route_order'] = float('nan')
ddds['range'] = float('nan')
ddds['range_status'] = np.where(coord_filter, 'calculating', 'out_of_range')
# ddds.loc[coord_filter, 'range_status'] = 'calculating'
ddds

,loc_name,address,city,state,zip,url,full_address,latitude,longitude,route_order,range_status,range
0,PANINI PETE'S,42 1/2 South Section St.,Fairhope,AL,36526,https://www.dinersdriveinsdiveslocations.com/p...,"42 1/2 South Section St., Fairhope, AL 36526",30.521934,-87.903195,NaN,out_of_range,NaN
1,GUMBO SHACK,212 Fairhope Ave.,Fairhope,AL,36532,https://www.dinersdriveinsdiveslocations.com/g...,"212 Fairhope Ave., Fairhope, AL 36532",30.522822,-87.905303,NaN,out_of_range,NaN
2,MANCI'S ANTIQUE CLUB,1715 Main St.,Daphne,AL,36526,https://www.dinersdriveinsdiveslocations.com/m...,"1715 Main St., Daphne, AL 36526",30.603415,-87.902965,NaN,out_of_range,NaN
3,SUNSET POINTE,831 N Section St,Fairhope,AL,36532,https://www.dinersdriveinsdiveslocations.com/s...,"831 N Section St, Fairhope, AL 36532",30.542483,-87.900378,NaN,out_of_range,NaN
4,THE HUMMINGBIRD WAY,351 George St,Mobile,AL,36604,https://www.dinersdriveinsdiveslocations.com/t...,"351 George St, Mobile, AL 36604",30.680789,-88.058675,NaN,out_of_range,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1383,BORN IN A BARN,100 E Ivinson Ave,Laramie,WY,82070,https://www.dinersdriveinsdiveslocations.com/b...,"100 E Ivinson Ave, Laramie, WY 82070",41.311986,-105.595931,NaN,calculating,NaN
1384,J'S PRARIE ROSE CAFE,410 S 2nd St,Laramie,WY,82070,https://www.dinersdriveinsdiveslocations.com/j...,"410 S 2nd St, Laramie, WY 82070",41.309707,-105.595324,NaN,calculating,NaN
1385,CROWBAR AND GRILL,202 S 2nd St,Laramie,WY,82070,https://www.dinersdriveinsdiveslocations.com/c...,"202 S 2nd St, Laramie, WY 82070",41.311898,-105.595118,NaN,calculating,NaN
1386,ALIBI WOOD FIRE,404 S 4th St,Laramie,WY,82070,https://www.dinersdriveinsdiveslocations.com/a...,"404 S 4th St, Laramie, WY 82070",41.309855,-105.592736,NaN,calculating,NaN


In [20]:
rdm_rt_pts = random.sample(list(enumerate(route_points)), len(route_points))
for idx, rdm_rt_pt in rdm_rt_pts:
    calcing = ddds['range_status'] == 'calculating'
    if ddds[calcing].shape[0] == 0:
        break

    ddddist = ddds.loc[calcing, ['latitude', 'longitude']].apply(range_calc, rdm_rt_pt=rdm_rt_pt, axis=1)
    ddds.loc[calcing, 'range_status'] = np.where(ddddist < search_distance, 'in_range', 'calculating')
    ddds.loc[ddds['range_status'] == 'in_range', 'route_order'] = idx
ddds.loc[ddds['range_status'] == 'calculating', 'range_status'] = 'out_of_range'
ddds_in_range = ddds[ddds['range_status'] == 'in_range']
ddds_in_range

,loc_name,address,city,state,zip,url,full_address,latitude,longitude,route_order,range_status,range
76,MAD GREEK'S DINER,72112 Baker Blvd,Baker,CA,92309,https://www.dinersdriveinsdiveslocations.com/m...,"72112 Baker Blvd, Baker, CA 92309",35.265910,-116.074773,12.0,in_range,NaN
86,EMMA JEAN'S HOLLAND BURGER CAFE,17143 N. D St.,Victorville,CA,92394,https://www.dinersdriveinsdiveslocations.com/e...,"17143 N. D St., Victorville, CA 92394",34.533413,-117.288951,12.0,in_range,NaN
110,POLKA RESTAURANT & CATERING,4112 Verdugo Road,Los Angeles,CA,90065,https://www.dinersdriveinsdiveslocations.com/p...,"4112 Verdugo Road, Los Angeles, CA 90065",34.125439,-118.231578,12.0,in_range,NaN
113,MOM'S TAMALES,3328 Pasadena Ave.,Los Angeles,CA,90031,https://www.dinersdriveinsdiveslocations.com/m...,"3328 Pasadena Ave., Los Angeles, CA 90031",34.083787,-118.213119,12.0,in_range,NaN
114,NICKEL DINER,524 South Main Street,Los Angeles,CA,90012,https://www.dinersdriveinsdiveslocations.com/n...,"524 South Main Street, Los Angeles, CA 90012",34.045985,-118.248779,12.0,in_range,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
950,QUEENS COMFORT,4009 30th Ave,Astoria,NY,11103,https://www.dinersdriveinsdiveslocations.com/q...,"4009 30th Ave, Astoria, NY 11103",40.763857,-73.914529,12.0,in_range,NaN
951,BRINDLE ROOM,277 E 10th St,New York,NY,10009,https://www.dinersdriveinsdiveslocations.com/b...,"277 E 10th St, New York, NY 10009",40.728295,-73.982704,12.0,in_range,NaN
952,BUN-KER VIETNAMESE,46-63 Metropolitan Ave,Ridgewood,NY,11385,https://www.dinersdriveinsdiveslocations.com/b...,"46-63 Metropolitan Ave, Ridgewood, NY 11385",40.713998,-73.921358,12.0,in_range,NaN
953,DON ANTONIO BY STARITA,309 W 50th St,New York,NY,10019,https://www.dinersdriveinsdiveslocations.com/d...,"309 W 50th St, New York, NY 10019",40.762658,-73.986645,12.0,in_range,NaN


In [25]:
def find_ddds_along_route(ddds, route_points, filter_window, search_distance):
    # ddds = all_ddd_locations#.drop(columns=['address', 'city', 'state', 'zip', 'url']).sample(20)################################
    sw_point, ne_point = filter_window
    coord_filter = (ddds['latitude'] > sw_point[0]) & (ddds['latitude'] < ne_point[0]) &\
                   (ddds['longitude'] > sw_point[1]) & (ddds['longitude'] < ne_point[1])
    
    ddds['route_order'] = float('nan')
    ddds['range'] = float('nan')
    ddds['range_status'] = np.where(coord_filter, 'calculating', 'out_of_range')
    rdm_rt_pts = random.sample(list(enumerate(route_points)), len(route_points))
    for idx, rdm_rt_pt in rdm_rt_pts:
        calcing = ddds['range_status'] == 'calculating'
        if ddds[calcing].shape[0] == 0:
            break
    
        ddddist = ddds.loc[calcing, ['latitude', 'longitude']].apply(range_calc, rdm_rt_pt=rdm_rt_pt, axis=1)
        ddds.loc[calcing, 'range_status'] = np.where(ddddist < search_distance, 'in_range', 'calculating')
        ddds.loc[ddds['range_status'] == 'in_range', 'route_order'] = idx
    ddds.loc[ddds['range_status'] == 'calculating', 'range_status'] = 'out_of_range'
    ddds_in_range = ddds[ddds['range_status'] == 'in_range']
    return ddds_in_range

In [26]:
find_ddds_along_route(all_ddd_locations, route_points, filter_window, search_distance)

,loc_name,address,city,state,zip,url,full_address,latitude,longitude,route_order,range,range_status
76,MAD GREEK'S DINER,72112 Baker Blvd,Baker,CA,92309,https://www.dinersdriveinsdiveslocations.com/m...,"72112 Baker Blvd, Baker, CA 92309",35.265910,-116.074773,48.0,NaN,in_range
86,EMMA JEAN'S HOLLAND BURGER CAFE,17143 N. D St.,Victorville,CA,92394,https://www.dinersdriveinsdiveslocations.com/e...,"17143 N. D St., Victorville, CA 92394",34.533413,-117.288951,48.0,NaN,in_range
110,POLKA RESTAURANT & CATERING,4112 Verdugo Road,Los Angeles,CA,90065,https://www.dinersdriveinsdiveslocations.com/p...,"4112 Verdugo Road, Los Angeles, CA 90065",34.125439,-118.231578,48.0,NaN,in_range
113,MOM'S TAMALES,3328 Pasadena Ave.,Los Angeles,CA,90031,https://www.dinersdriveinsdiveslocations.com/m...,"3328 Pasadena Ave., Los Angeles, CA 90031",34.083787,-118.213119,48.0,NaN,in_range
114,NICKEL DINER,524 South Main Street,Los Angeles,CA,90012,https://www.dinersdriveinsdiveslocations.com/n...,"524 South Main Street, Los Angeles, CA 90012",34.045985,-118.248779,48.0,NaN,in_range
...,...,...,...,...,...,...,...,...,...,...,...,...
950,QUEENS COMFORT,4009 30th Ave,Astoria,NY,11103,https://www.dinersdriveinsdiveslocations.com/q...,"4009 30th Ave, Astoria, NY 11103",40.763857,-73.914529,48.0,NaN,in_range
951,BRINDLE ROOM,277 E 10th St,New York,NY,10009,https://www.dinersdriveinsdiveslocations.com/b...,"277 E 10th St, New York, NY 10009",40.728295,-73.982704,48.0,NaN,in_range
952,BUN-KER VIETNAMESE,46-63 Metropolitan Ave,Ridgewood,NY,11385,https://www.dinersdriveinsdiveslocations.com/b...,"46-63 Metropolitan Ave, Ridgewood, NY 11385",40.713998,-73.921358,48.0,NaN,in_range
953,DON ANTONIO BY STARITA,309 W 50th St,New York,NY,10019,https://www.dinersdriveinsdiveslocations.com/d...,"309 W 50th St, New York, NY 10019",40.762658,-73.986645,48.0,NaN,in_range
